In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
URL="https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"

In [3]:
HEADERS=({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36','Accept-Language':'en-US, en;q=0.5'})

In [4]:
URLS=[]

In [5]:
webpage=requests.get(URL,headers=HEADERS)

In [6]:
soup=BeautifulSoup(webpage.content,'html.parser')

In [7]:
links=soup.find_all('a',class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')

In [8]:
for i in links:
        lin=i.get('href')
        product_link='https://www.amazon.in'+lin
        URLS.append(product_link)

In [9]:
# Function to extract Product Title
def get_title(soup):

    try:
        title = soup.find("span", attrs={"id":'productTitle'}).text.strip()

    except AttributeError:
        title= ""

    return title


# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price


# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating



#Function to get Number of Reviews

def get_review(soup):
    try:
        review=soup.find('span',attrs={'id':'acrCustomerReviewText'}).text
    except:
        review=""
    return review


#Function to get Product Description

def get_description(soup):
    try: 
        desc=soup.find('ul',attrs={'class':'a-unordered-list a-vertical a-spacing-mini'}).text.replace('<br>','')
    except:
        desc=""
    
    return desc

#Function to get ASIN number
def get_asin(soup):
    try:
        tab=soup.find('table',{'id':'productDetails_detailBullets_sections1'})
        asin_number=tab.find_all('td',{'class':'a-size-base prodDetAttrValue'})[0].text.strip()
    except AttributeError:
        try:
            asin_data=soup.find('ul',{'class':'a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list'})
            asin_number=asin_data.find_all('li')[3].find_all('span')[2].text
        except:
            asin_number=''
    
    return asin_number

#Function to get Manufacturer Name
def get_manufacturer(soup):
    try:
        tab=soup.find('table',{'id':'productDetails_techSpec_section_1'})
        manufac_name=tab.find_all('td',{'class':'a-size-base prodDetAttrValue'})[1].text.replace('\u200e',"").strip()
    except AttributeError:
        try:
            manufac_data=soup.find('ul',{'class':'a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list'})
            manufac_name=manufac_data.find_all('li')[2].find_all('span')[2].text
        except:
            manufac_name=''
        
    return manufac_name




In [10]:
#Loop to go to next page and gettting the product links

count=1 #Initializing with value 1 and getting the data upto 20th page 
while True:
    count+=1
    np=soup.find('a',class_='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator').get('href')
    cnp='https://www.amazon.in'+np

    URL=cnp
    webpage=requests.get(URL,headers=HEADERS)
    soup=BeautifulSoup(webpage.content,'html.parser')
    links=soup.find_all('a',class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    #Adding all the links of products from each page
    for i in links:
        lin=i.get('href')
        product_link='https://www.amazon.in'+lin
        URLS.append(product_link)
    
    if count>=20:
        break
# print(URLS)


In [11]:
product_url=[]
product_name=[]
price=[]
rating=[]
review=[]
pro_desc=[]
pro_asin=[]
pro_manufact=[]

In [15]:
#Travesing through all the product links and collecting all the data and storing them in lists
count=0  # Count variable used to break the iteration after collecting more than 200 data items
for i in URLS:
    count+=1
    new_webpage=requests.get(i,headers=HEADERS)
    new_soup=BeautifulSoup(new_webpage.content,'html.parser')
    product_url.append(i)
    product_name.append(get_title(new_soup))
    price.append(get_price(new_soup))
    rating.append(get_rating(new_soup))
    review.append(get_review(new_soup))
    pro_desc.append(get_description(new_soup))
    pro_asin.append(get_asin(new_soup))
    pro_manufact.append(get_manufacturer(new_soup))
    
    if count>=200:
        break
    

In [16]:
dict={'PRODUCT URL':product_url,'NAME':product_name,'PRICE':price,'RATING':rating,'REVIEWS': review, 'DESCRIPTION': pro_desc,'ASIN':pro_asin,"MANUFACTURER":pro_manufact}
df=pd.DataFrame(dict)
df.to_csv('AmazonScrapeData.csv', encoding='utf-8')
df.head(20)

,PRODUCT URL,NAME,PRICE,RATING,REVIEWS,DESCRIPTION,ASIN,MANUFACTURER
0,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,"Targus 12.5-15.6"" Geolite Advanced TSB96201GL ...",,4.4 out of 5 stars,156 ratings,Dedicated laptop compartment with multi-fit ...,B07JQJTRHY,"Targus, TARGUS ASIA PACIFIC LIMITED, UNIT 1202..."
1,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,uppercase Matrix Professional Anti-theft Lapto...,,4.2 out of 5 stars,12 ratings,Care Instructions: Wipe with Damp Cloth H...,B0BSH4H4DB,Acefour Accessories Pvt. Ltd.
2,https://www.amazon.in/American-Tourister-AMT-S...,American Tourister 32 Ltrs Black Casual Backpa...,,4.0 out of 5 stars,"52,312 ratings","Laptop Compatibility: No, Strap Type: Adjust...",B07CJCGM1M,0.5 Kilograms
3,https://www.amazon.in/Wesley-Milestone-Waterpr...,Wesley Milestone 2.0 Casual Waterproof Laptop ...,,4.3 out of 5 stars,"9,817 ratings",30L Capacity: The Backpack has a padded lapt...,B084JGJ8PF,"Wesley, Longani Trading Company, F-82, Sector-..."
4,https://www.amazon.in/Skybags-Brat-Black-Casua...,Skybags Brat Black 46 Cms Casual Backpack,,4.1 out of 5 stars,"3,777 ratings",Combination of functional & safety features ...,B08Z1HHHTD,VIP Industries Ltd
5,https://www.amazon.in/Lavie-Sport-Duffle-Lugga...,Lavie Sport Lino Large Size 63 cms Wheel Duffl...,,3.9 out of 5 stars,"6,307 ratings",The Lino wheel Duffle Bag from Lavie Sport i...,B097RJ22Q3,Bagzone Lifestyle Pvt Ltd
6,https://www.amazon.in/ADISA-Laptop-Backpack-Of...,ADISA 15.6 inch Laptop Backpack Office Bag Col...,,3.8 out of 5 stars,567 ratings,Material: Water Resistant Light-Weight Polye...,B09TPX22NF,ADISA
7,https://www.amazon.in/TRUE-HUMAN-Anti-Theft-ba...,TRUE HUMAN® Emperor Anti-Theft backpack With U...,,3.7 out of 5 stars,"1,829 ratings",Ease of Access-Multi purpose pockets Super s...,LEGEND-1,B09QJ7ZF2Y
8,https://www.amazon.in/Half-Moon-Waterproof-Bac...,Half Moon 35 L Waterproof Laptop College Schoo...,,3.9 out of 5 stars,"15,254 ratings",SPACIOUS AND LIGHTWEIGHT: Light weight and s...,B085MHDJ93,Amazing Bags
9,https://www.amazon.in/GEAR-Black-Blue-Backpack...,Gear CarryOn 16L Foldable Water Resistant Scho...,,3.7 out of 5 stars,"6,058 ratings","Outer Material: Polyester, Color: Black and ...",Gear Merchandise PVT LTD,17 December 2015
